# Setup

In [160]:
import numpy as np

with open('input.txt') as f:
    lines = f.readlines()

offset_from_char = {
    '^': (-1, 0),
    '>': (0, 1),
    'v': (1, 0),
    '<': (0, -1)
}

blank_line = lines.index('\n')
board, movements = lines[:blank_line], lines[blank_line+1:]
board = np.array([[c for c in line.strip()] for line in board])
movements = np.array([[c for c in line.strip()] for line in movements]).flatten()

SHAPE = board.shape

# Part 1

In [ ]:
start_position = tuple(*zip(*np.where(board == '@')))
box_positions = list(zip(*np.where(board == 'O')))
wall_positions = list(zip(*np.where(board == '#')))

def try_move(position: tuple[int, int], direction: str) -> tuple[int, int]:
    offset = offset_from_char[direction]
    check_position = (position[0] + offset[0], position[1] + offset[1])
    if check_position in wall_positions:
        return position
    
    if check_position in box_positions:
        if try_move(check_position, direction) == check_position:
            return position
        else:
            box_positions.remove(check_position)
            box_positions.append((check_position[0] + offset[0], check_position[1] + offset[1]))
    
    return check_position
    
def calculate_score(at_pos: tuple[int, int], box_pos: list[tuple[int, int]], wall_pos: list[tuple[int, int]]) -> tuple[int, np.ndarray]:
    board_to_show = np.ndarray(SHAPE, dtype='<U1')
    board_to_show.fill('.')
    for y, x in wall_pos:
        board_to_show[y, x] = '#'
    for y, x in box_pos:
        board_to_show[y, x] = 'O'
    board_to_show[at_pos[0], at_pos[1]] = '@'
    
    box_positions = list(zip(*np.where(board_to_show == 'O')))
    score = 0
    for y, x in box_positions:
        score += 100 * y + x
    
    return score, board_to_show
    
for move in movements:
    start_position = try_move(start_position, move)
    
score, board_to_show = calculate_score(start_position, box_positions, wall_positions)
# print(board_to_show)
print(score)

# Part 2
Today is really scuffed. I bet if I wanted to, I could clean up the Part 2 solution and have Part 1 and 2 use the same approach. But it works. It takes quite a while to run, hopefully because of the many print statements and not because my code is bad (It did print over 5 Million characters :) )

In [ ]:
import itertools
SHAPE = (SHAPE[0], SHAPE[1] * 2)
start_position = tuple(*zip(*np.where(board == '@')))
start_position = (start_position[0], 2*start_position[1])
box_positions = [[(y, 2*x), (y, 2*x+1)] for y, x in zip(*np.where(board == 'O'))]
wall_positions = list(itertools.chain(*[[(y, 2*x), (y, 2*x+1)] for y, x in zip(*np.where(board == '#'))]))

def get_box_id(pos: tuple[int, int]) -> int:
    return next(i for i, dict_pos in box_id.items() if pos == dict_pos)

buddy_boxes = {left: right for left, right in box_positions}
box_positions: list[tuple[int, int]] = list(itertools.chain(*box_positions))
box_id: dict[tuple[int, int], int] = {i: pos for i, pos in enumerate(box_positions)}
buddy_boxes: dict[int, int] = {get_box_id(left): get_box_id(right) for left, right in buddy_boxes.items()}

def get_buddy_box(id: int) -> int:
    if id in buddy_boxes:
        return buddy_boxes[id]
    else:
        return next(key for key, value in buddy_boxes.items() if value == id)

def update_positions_after_move_check(boxes: set[int], direction: str) -> None:
    offset = offset_from_char[direction]
    for id in boxes:
        old_pos = box_id[id]
        new_pos = (old_pos[0] + offset[0], old_pos[1] + offset[1])
        print(f"Moving box {id} from {old_pos} to {new_pos}")
        box_id[id] = new_pos   
                
positions_checked: dict[tuple[int, int], bool] = {}
boxes_moved: set[int] = set()

def calculate_score_2(at_pos: tuple[int, int], box_pos: list[tuple[int, int]], wall_pos: list[tuple[int, int]]) -> tuple[int, np.ndarray]:
    board_to_show = np.ndarray(SHAPE, dtype='<U1')
    board_to_show.fill('.')
    for y, x in wall_pos:
        board_to_show[y, x] = '#'
    for y, x in box_pos:
        board_to_show[y, x] = 'O'
    board_to_show[at_pos[0], at_pos[1]] = '@'
    
    score = 0
    
    for id, buddy_box_id in buddy_boxes.items():
        box_pos = box_id[id]
        buddy_box_pos = box_id[buddy_box_id]
        
        score += 100 *box_pos[0] + min(box_pos[1], buddy_box_pos[1])
    
    return score, board_to_show

def try_move_2_x(position: tuple[int, int], direction: str) -> tuple[int, int]:
    offset = offset_from_char[direction]
    check_position = (position[0] + offset[0], position[1] + offset[1])
    if check_position in wall_positions:
        return position
    
    if check_position in box_id.values():
        if try_move_2_x(check_position, direction) == check_position:
            return position
        else:
            id = get_box_id(check_position)
            print(f"Moving box {id} from {check_position} to {check_position[0] + offset[0], check_position[1] + offset[1]}")
            box_id[id] = (check_position[0] + offset[0], check_position[1] + offset[1])
    
    return check_position

def try_move_2(position: tuple[int, int], direction: str, top_level_flag = False) -> tuple[tuple[int, int], bool]:
    if direction not in ('^', 'v'):
        print(f"from pos {position}, dir {direction} using try_move_2_x")
        ret_pos = try_move_2_x(position, direction)
        if ret_pos == position:
            return position, False
        return ret_pos, True
    
    if position in positions_checked:
        print(f"From pos {position}, dir {direction} Checking {position}, already checked, {positions_checked[position]}")
        return position, positions_checked[position]
    
    offset = offset_from_char[direction]
    check_position = (position[0] + offset[0], position[1] + offset[1])
    if check_position in wall_positions:
        print(f"From pos {position}, dir {direction} Checking {check_position}, wall")
        return position, False
    
    if check_position in box_id.values():
        check_position_box_id = get_box_id(check_position)
        buddy_box_id = get_buddy_box(check_position_box_id)
        buddy_box_pos = box_id[buddy_box_id]
        
        print(f"From pos {position}, dir {direction} Checking {check_position}, id: {check_position_box_id} with buddy {buddy_box_pos}, id {buddy_box_id}")
        
        _, can_buddy_box_move = try_move_2(buddy_box_pos, direction)
        _, can_direct_move = try_move_2(check_position, direction)
                
        positions_checked[check_position] = can_direct_move
        positions_checked[buddy_box_pos] = can_buddy_box_move
        
        if can_buddy_box_move and can_direct_move:
            boxes_moved.add(check_position_box_id)
            boxes_moved.add(buddy_box_id)
            if top_level_flag:
                print(f"Updating positions after move")
                update_positions_after_move_check(boxes_moved, direction)
                positions_checked.clear()
                boxes_moved.clear()
                
            return check_position, True
        else:
            if top_level_flag:
                positions_checked.clear()
                boxes_moved.clear()
            return position, False
    else:
        print(f"From pos {position}, dir {direction} Checking {check_position}, empty") 
    
    return check_position, True

_, board_to_show = calculate_score_2(start_position, box_id.values(), wall_positions)
print(board_to_show)
for move in movements:
    start_position, _ = try_move_2(start_position, move, top_level_flag=True)
    score, board_to_show = calculate_score_2(start_position, box_id.values(), wall_positions)
    print(board_to_show)
score